<a href="https://colab.research.google.com/github/EmiliaFidler/Intro_to_Comp_Ling_WS24/blob/main/homeexercise3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Home Exericse 3: Hyperparameters and Evaluation
In this third home exercise, you will use the knowledge from Tutorial 4 to experiment with hyperparameters, create a test set, and evaluate your final model on the created test set.

In this notebook, please complete all instructions starting with 👋 ⚒ in the code cell after the sign or provide your analysis in the text cell after the sign.

## **Distilbert: Hyperparameters and Evaluation**

Use the code of Tutorial 4 to load and fine-tune the `distilbert-base-cased`model on the small subset of the `imdb`Movie Review Dataset. For convenience, the code of Tutorial 4 required for this exercise is already provided in the code cells below.

👋 ⚒ When creating the dataset splits in the code cell below, additionally create a test set to be used after thet training. Make sure that your test set does not contain any of the sentences contained in the training or validation set and is approximately of the same size as the validation set.

In [3]:
!pip install transformers
!pip install datasets
!pip install evaluate
!pip install accelerate --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.0 MB/s eta 0:00:00


In [4]:
from datasets import load_dataset, DatasetDict
from transformers import DataCollatorWithPadding, AutoTokenizer

imdb_dataset = load_dataset("imdb")
# we had loaded the imdb dataset already above - if not, outcomment this line
# Make sure you have the right tokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-cased")


# Just take the first 50 tokens for speed on CPU
def truncate(example):
    return {
        'text': " ".join(example['text'].split()[:50]),
        'label': example['label']
    }



# Take 128 random examples for train and 32 validation
small_imdb_dataset = DatasetDict(
    train=imdb_dataset['train'].shuffle(seed=24).select(range(128)).map(truncate),
    val=imdb_dataset['train'].shuffle(seed=24).select(range(128, 160)).map(truncate),
    test=imdb_dataset['test'].shuffle(seed=24).select(range(160, 192)).map(truncate)
)

def tokenize_function(examples):
    return tokenizer(examples["text"], padding=True, truncation=True)

small_tokenized_dataset = small_imdb_dataset.map(tokenize_function, batched=True, batch_size=16)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

...

Map:   0%|          | 0/32 [00:00<?, ? examples/s]

Map:   0%|          | 0/32 [00:00<?, ? examples/s]

Map:   0%|          | 0/128 [00:00<?, ? examples/s]

Map:   0%|          | 0/32 [00:00<?, ? examples/s]

Map:   0%|          | 0/32 [00:00<?, ? examples/s]

👋 ⚒ For this exercise, we will use the Hugging Face Trainer class to play with hyperparamters. Try to find a set of hyperparameter settings that achieves the highest possilbe accuracy on the **validation set** with the small dataset and model in this setup.

**Optional:** If you want to follow a more systematic route, feel free to use available frameworks for hyperparameter optimization, such as [Optuna](https://optuna.org/).

In [ ]:
!pip install optuna

  Using cached optuna-4.1.0-py3-none-any.whl.metadata (16 kB)
  Using cached alembic-1.14.0-py3-none-any.whl.metadata (7.4 kB)
  Using cached colorlog-6.9.0-py3-none-any.whl.metadata (10 kB)
  Using cached Mako-1.3.6-py3-none-any.whl.metadata (2.9 kB)
Using cached optuna-4.1.0-py3-none-any.whl (364 kB)
Using cached alembic-1.14.0-py3-none-any.whl (233 kB)
Using cached colorlog-6.9.0-py3-none-any.whl (11 kB)
Using cached Mako-1.3.6-py3-none-any.whl (78 kB)


In [5]:
import numpy as np
import evaluate
from transformers import TrainingArguments, Trainer
from transformers import AutoModelForSequenceClassification
from transformers import EarlyStoppingCallback

model = AutoModelForSequenceClassification.from_pretrained('distilbert/distilbert-base-cased', num_labels=2)
accuracy = evaluate.load("accuracy")

arguments = TrainingArguments(
    output_dir="sample_cl_trainer", # where to put everything that is trained
    per_device_train_batch_size=8,  # amount of sentences model looks at before anything is updated
    per_device_eval_batch_size=8,
    logging_steps=8,                # how many times results are logged/track/output
    num_train_epochs=10,
    eval_strategy="epoch",          # run validation at the end of each epoch
    save_strategy="epoch",
    learning_rate=1e-5,             # influences how weight changes in function, determines how big a step you take in valley
    weight_decay=0.005,              # sum of weights added to loss function, regulization method, penalizes large weights
    load_best_model_at_end=True,
    report_to='none',
    seed=224
)

def compute_metrics(eval_pred):
    """Called at the end of validation. Gives accuracy"""
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    # calculates the accuracy
    return accuracy.compute(predictions=predictions, references=labels)


trainer = Trainer(
    model=model,
    args=arguments,
    train_dataset=small_tokenized_dataset['train'],
    eval_dataset=small_tokenized_dataset['val'], # change to test when you do your final evaluation!
    processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

trainer.add_callback(EarlyStoppingCallback(early_stopping_patience=2))

model.safetensors:   0%|          | 0.00/263M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.682200,0.693174,0.468750
2,0.684100,0.688901,0.468750
3,0.663700,0.672419,0.500000
4,0.612400,0.644293,0.781250
5,0.562400,0.607863,0.812500
6,0.462400,0.553763,0.843750
7,0.375600,0.513987,0.812500
8,0.314200,0.478871,0.875000
9,0.267300,0.463749,0.875000
10,0.272500,0.459768,0.843750


TrainOutput(global_step=160, training_loss=0.5006450787186623, metrics={'train_runtime': 1239.8815, 'train_samples_per_second': 1.032, 'train_steps_per_second': 0.129, 'total_flos': 36337463293824.0, 'train_loss': 0.5006450787186623, 'epoch': 10.0})

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
results = trainer.predict(small_tokenized_dataset['val'])
print(results)

PredictionOutput(predictions=array([[0.13421392, 0.12798628],
       [0.12095223, 0.11954887],
       [0.17105228, 0.12995353],
       [0.15801448, 0.13184753],
       [0.10921323, 0.12416728],
       [0.12570968, 0.12112349],
       [0.10696004, 0.14420581],
       [0.11631259, 0.11794458],
       [0.15121919, 0.0889772 ],
       [0.1495204 , 0.11558668],
       [0.14313394, 0.13188398],
       [0.1387461 , 0.12331152],
       [0.14149424, 0.11447348],
       [0.1478344 , 0.12460323],
       [0.15348402, 0.12031263],
       [0.11492486, 0.13474976],
       [0.11108047, 0.11179354],
       [0.13802245, 0.11373404],
       [0.13766065, 0.15556134],
       [0.14539611, 0.11064982],
       [0.09936307, 0.13165972],
       [0.12217208, 0.13019106],
       [0.17329057, 0.13361736],
       [0.1384731 , 0.13353653],
       [0.12050661, 0.12344055],
       [0.10287175, 0.12043194],
       [0.12166733, 0.1076322 ],
       [0.15191486, 0.1471841 ],
       [0.11044995, 0.10262597],
       [0.0994

👋 ⚒ Change the following code cell in a way that not only a single sentence is evaluated on your trained model (!make sure to use the correct checkpoint!) but the evaluation is performaned on the entire newly created test set.

This might also be a good occassion to get familiar with the [Hugging Face documentation and tutorials](https://huggingface.co/docs/transformers/index).

In [20]:
import torch

test_str = "It was so not great!"

fine_tuned_model = AutoModelForSequenceClassification.from_pretrained("drive/MyDrive/checkpoint-144")
model_inputs = tokenizer(test_str, return_tensors="pt")
prediction = torch.argmax(fine_tuned_model(**model_inputs).logits)
print(["NEGATIVE", "POSITIVE"][prediction])

NEGATIVE


In [9]:
test_dataset = small_imdb_dataset['test']

test_results = trainer.predict(small_tokenized_dataset['test'])
print("Test set evaluation results:", test_results)

Test set evaluation results: PredictionOutput(predictions=array([[0.16713774, 0.1377847 ],
       [0.14085874, 0.09917316],
       [0.17267935, 0.11417043],
       [0.09764283, 0.12184554],
       [0.1429949 , 0.12962246],
       [0.14720464, 0.1348001 ],
       [0.13529262, 0.14095561],
       [0.19194353, 0.10265684],
       [0.17908093, 0.12335928],
       [0.14415044, 0.14294975],
       [0.14766079, 0.13525839],
       [0.11834985, 0.1274413 ],
       [0.1199066 , 0.119968  ],
       [0.14462963, 0.12059249],
       [0.13082173, 0.13267714],
       [0.11749665, 0.1246297 ],
       [0.14341906, 0.14327544],
       [0.11570236, 0.14298153],
       [0.13486728, 0.13108075],
       [0.12300302, 0.10221151],
       [0.15819202, 0.12940389],
       [0.15419038, 0.13216075],
       [0.13394621, 0.08424798],
       [0.14456637, 0.14592102],
       [0.13893513, 0.11288892],
       [0.1335062 , 0.1276733 ],
       [0.12906098, 0.12549496],
       [0.14348769, 0.14246446],
       [0.12022826

In [18]:
tokenized_test_dataset = test_dataset.map(
    tokenize_function,
    batched=True,
    batch_size=16
)

def compute_metrics(eval_pred):
    """Called at the end of validation. Gives accuracy"""
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    # calculates the accuracy
    return accuracy.compute(predictions=predictions, references=labels)

# new trainer for evaluation
trainer = Trainer(
    model=fine_tuned_model,
    args=arguments,
    train_dataset=small_tokenized_dataset['train'],
    eval_dataset=small_tokenized_dataset['test'], # change to test when you do your final evaluation!
    processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

test_results = trainer.evaluate(eval_dataset=tokenized_test_dataset)
print("Test set evaluation results:", test_results)

Test set evaluation results: {'eval_loss': 0.5175219774246216, 'eval_model_preparation_time': 0.003, 'eval_accuracy': 0.84375, 'eval_runtime': 10.2281, 'eval_samples_per_second': 3.129, 'eval_steps_per_second': 0.391}


In [23]:
from sklearn.metrics import matthews_corrcoef, accuracy_score, precision_recall_fscore_support

In [25]:
def compute_metrics(predictions):
    labels = predictions.label_ids
    predictions = np.argmax(predictions.predictions, axis=1)

    mcc = matthews_corrcoef(labels, predictions) #
    acc = accuracy_score(labels, predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average="binary")

    return {
        "mcc": mcc,
        "accuracy": acc,
        "precision": precision,
        "recall": recall,
        "f1": f1,
    }

test_results = trainer.evaluate(small_tokenized_dataset["test"], metric_key_prefix="test")
predictions = trainer.predict(small_tokenized_dataset["test"])

metrics = compute_metrics(predictions)

print("Evaluation Results:")
print(f"Test Loss: {test_results['test_loss']:.4f}")
print(f"Matthews Correlation Coefficient (MCC): {metrics['mcc']:.4f}")
print(f"Accuracy: {metrics['accuracy']:.4f}")
print(f"Precision: {metrics['precision']:.4f}")
print(f"Recall: {metrics['recall']:.4f}")
print(f"F1 Score: {metrics['f1']:.4f}")

Evaluation Results:
Test Loss: 0.5175
Matthews Correlation Coefficient (MCC): 0.6864
Accuracy: 0.8438
Precision: 0.8824
Recall: 0.8333
F1 Score: 0.8571
